In [ ]:
# model_converter.py

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from optimum.quanto import Calibration, QuantizedModelForCausalLM, qfloat8, qint4, qint8

def convert_model_to_float8(model_id: str, output_dir: str, push_to_hub: float=False):
    """
    모델을 8비트로 변환하여 저장하는 함수
    """
    try:
        # 모델과 토크나이저 로드
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,  # 또는 적절한 dtype 사용
            device_map="auto",
        )
        qmodel=QuantizedModelForCausalLM.quantize(model, weights=qfloat8, activations=qint8)
        qmodel.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f"모델이 성공적으로 8비트로 변환되어 '{output_dir}'에 저장되었습니다.")

        if push_to_hub:
            model_name=f"{model_id.split('/', -1)}-8bit"
            qmodel.push_to_hub(f"{model_name}-8bit")
            print(f"모델이 성공적으로 8비트로 변환되어 '{model_name}'에 푸시되었습니다.")
        
        return True
    except Exception as e:
        print(f"모델 변환 중 오류 발생: {e}")
        return False

In [24]:
from optimum.quanto import freeze, qint8, quantize
from diffusers import StableDiffusionXLPipeline
from safetensors import serialize_file
import torch

pipeline = StableDiffusionXLPipeline.from_pretrained(
    "cagliostrolab/animagine-xl-3.1", 
    torch_dtype=torch.bfloat16,
).to("cpu")
quantize(pipeline.unet, weights=qint8)
freeze(pipeline.unet)
quantize(pipeline.text_encoder, weights=qint8)
freeze(pipeline.text_encoder)
quantize(pipeline.text_encoder_2, weights=qint8)
freeze(pipeline.text_encoder_2)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [25]:
pipeline.save_pretrained("diffusion_models/animagine-xl-3.1-int8", safe_serialization=True)

In [1]:
from optimum.quanto import QuantizedDiffusersModel
from diffusers import StableDiffusionXLPipeline
from safetensors import serialize_file
import torch

class QuantizedStableDiffusionXLPipeline(QuantizedDiffusersModel):
    base_class = StableDiffusionXLPipeline

In [2]:
# safetensors 파일에서 파이프라인 로드
pipeline_loaded = QuantizedStableDiffusionXLPipeline.from_pretrained(
    "diffusion_models/animagine-xl-3.1-int8",
    torch_dtype=torch.float16,
).to("mps")  # 또는 "cuda"

ValueError: No quantization map found in diffusion_models/animagine-xl-3.1-int8: is this a quantized model ?

In [18]:
from safetensors.torch import load_file
unet_safetensors_path='diffusion_models/animagine-xl-3.1-int8/unet/diffusion_pytorch_model.safetensors'

unet_state_dict_loaded = load_file(unet_safetensors_path)
pipeline_loaded.unet.load_state_dict(unet_state_dict_loaded)

RuntimeError: Error(s) in loading state_dict for UNet2DConditionModel:
	Missing key(s) in state_dict: "conv_in.weight", "time_embedding.linear_1.weight", "time_embedding.linear_2.weight", "add_embedding.linear_1.weight", "add_embedding.linear_2.weight", "down_blocks.0.resnets.0.conv1.weight", "down_blocks.0.resnets.0.time_emb_proj.weight", "down_blocks.0.resnets.0.conv2.weight", "down_blocks.0.resnets.1.conv1.weight", "down_blocks.0.resnets.1.time_emb_proj.weight", "down_blocks.0.resnets.1.conv2.weight", "down_blocks.0.downsamplers.0.conv.weight", "down_blocks.1.attentions.0.proj_in.weight", "down_blocks.1.attentions.0.transformer_blocks.0.attn1.to_q.weight", "down_blocks.1.attentions.0.transformer_blocks.0.attn1.to_k.weight", "down_blocks.1.attentions.0.transformer_blocks.0.attn1.to_v.weight", "down_blocks.1.attentions.0.transformer_blocks.0.attn1.to_out.0.weight", "down_blocks.1.attentions.0.transformer_blocks.0.attn2.to_q.weight", "down_blocks.1.attentions.0.transformer_blocks.0.att

In [9]:

prefixed_unet_state_dict = {f"unet.{k}": v.cpu() for k, v in unet_state_dict.items()}
prefixed_text_encoder_state_dict = {f"text_encoder.{k}": v.cpu() for k, v in text_encoder_state_dict.items()}
prefixed_text_encoder_2_state_dict = {f"text_encoder_2.{k}": v.cpu() for k, v in text_encoder_state_dict.items()}

# 전체 state_dict 결합
combined_state_dict = {**prefixed_unet_state_dict, **prefixed_text_encoder_state_dict}

combined_state_dict_2 = {**combined_state_dict, **prefixed_text_encoder_2_state_dict}

In [10]:
from safetensors.torch import save_file

# safetensors 파일로 저장
safetensors_path = "diffusion_models/animagine-xl-3.1-int8.safetensors"
save_file(combined_state_dict_2, safetensors_path)

RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'text_encoder.text_model.embeddings.token_embedding.weight', 'text_encoder_2.text_model.embeddings.token_embedding.weight'}, {'text_encoder_2.text_model.embeddings.position_embedding.weight', 'text_encoder.text_model.embeddings.position_embedding.weight'}, {'text_encoder_2.text_model.encoder.layers.0.self_attn.k_proj.weight._data', 'text_encoder.text_model.encoder.layers.0.self_attn.k_proj.weight._data'}, {'text_encoder_2.text_model.encoder.layers.0.self_attn.k_proj.weight._scale', 'text_encoder.text_model.encoder.layers.0.self_attn.k_proj.weight._scale'}, {'text_encoder_2.text_model.encoder.layers.0.self_attn.k_proj.bias', 'text_encoder.text_model.encoder.layers.0.self_attn.k_proj.bias'}, {'text_encoder_2.text_model.encoder.layers.0.self_attn.k_proj.input_scale', 'text_encoder.text_model.encoder.layers.0.self_attn.k_proj.input_scale'}, {